## Import SqliteCloudClient and SqliteCloudAccount

SqliteCloudAccount is the class rapresenting your auth data for SqliteCloud

SqliteCloudClient is the class managing the connection for you

In [ ]:
from sqlitecloud.conn_info import user,password,host,db_name,port
from sqlitecloud.client import SqliteCloudClient, SqliteCloudAccount

## Init a connection

Initialize the client with account connection info

In [ ]:
account = SqliteCloudAccount(user, password, host, db_name, port)
client = SqliteCloudClient(cloud_account=account)
conn = client.open_connection()

Then execute the query

In [ ]:
query = "select * from employees;"
result = client.exec_query(query, conn)

The result is an iterable

In [ ]:
for r in result:
    print(r)

Whe you are done clean up the connection

In [ ]:
client.disconnect(conn)


You can bind values to parametric queries: you can pass parameters as positional values in an array

In [ ]:
new_connection = client.open_connection()
result = client.exec_statement("select * from employees where emp_id = ?", [1],conn=new_connection)
for r in result:
    print(r)
client.disconnect(conn)